## Install required libraries


In [ ]:
!pip install diffusers==0.26.3
!pip install transformers scipy ftfy accelerate

In [ ]:
import torch
from diffusers import DiffusionPipeline, LCMScheduler
from PIL import Image, ImageFilter
import numpy as np
import random

## Create diffusion pipeline

In [ ]:
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0")

In [ ]:
pipe.load_lora_weights("nerijs/pixel-art-xl")

In [ ]:
pipe.to(device="cuda", dtype=torch.float16)

## Create prompts

In [ ]:
prompt = "cobblestone background, simple, flat colors, pixel art, video game, tile"
negative_prompt = "blurry, photorealistic, 3d render, realistic, character"

## Create seed

In [ ]:
# 4382350725
# 4382342934
# 4382339547
# 4382337597
# 438233954
# 4382356118

base_seed = 4382339547
seed_offset = random.randint(0, 20000)

seed = base_seed + seed_offset

print(f"seed {seed}")

## Generate Images

In [ ]:
image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=20,
        guidance_scale=1.5,
        generator=torch.Generator().manual_seed(seed)
    ).images[0]

image

In [ ]:
image.resize((128, 128), Image.NEAREST).filter(ImageFilter.BoxBlur(radius=0))

image

## Downscale to 32x32

In [ ]:
def resize_to_any(pimage, size=32):
  img = pimage.resize((size, size), Image.NEAREST)
  img = img.filter(ImageFilter.BoxBlur(radius=0))
  return img

In [ ]:
# cropped_image for character
# image for background
resized_image = resize_to_any(image, 32)
resized_image